### Imports

In [1]:
import syft as sy
import torch
from tools import models, datasets
import numpy as np
import pandas as pd
import opacus
from tools.utils import train, test

### Loading the data

In [2]:
# In the first run download=True has to be passed to this function in order to automatically dowload the data!
train_ds, test_ds, val_ds = datasets.Loader.load_MedNIST(sample_size=0.04, test_size=0.0872, val_size=0.125)
train_data, train_labels = train_ds.as_tensor()
test_data, test_labels = test_ds.as_tensor()
val_data, val_labels = val_ds.as_tensor()

print(len(train_data))
print(len(test_data))
print(len(val_data))

1858
206
294


### Parameters and constants

In [3]:
# Constants for tracking purposes
MODEL = 'Deep2DNet'
DATASET = 'MedNIST'
TRACKING = False # Whether or not this run should be tracked in the results.csv
DP = True # Whether or not Differential Privacy should be applied

BATCH_SIZE = 100
EPOCHS = 30

DELTA = 0.0001 # Set to be less then the inverse of the training dataset (from https://opacus.ai/tutorials/building_image_classifier)

# Parameters for training
length = len(train_data)
SAMPLE_SIZE = length - length % BATCH_SIZE # NOTE: Current implementation only trains data in multiples of batch size. So BATCH_SIZE % LENGTH amount of data will not be used for training.
SAMPLE_RATE = BATCH_SIZE / SAMPLE_SIZE

### Training and testing

In [4]:
def run(learning_rate, noise_multiplier, max_grad_norm):
    # Getting model
    model = models.Deep2DNet(torch)

    # Setting device to train on
    cuda_available = torch.cuda.is_available()
    if cuda_available:
        device = torch.device('cuda:0')
        model.cuda(device)
    else:
        device = torch.device('cpu')
        model.cpu()

    # Optimizer and Loss Function
    params = model.parameters()
    optim = torch.optim.Adam(params=params, lr=learning_rate)
    loss_function = torch.nn.CrossEntropyLoss()

    # Setting up Differential Privacy Engine
    if DP:
        privacy_engine = opacus.privacy_engine.PrivacyEngine(
            model.real_module, sample_rate=SAMPLE_RATE,
            noise_multiplier=noise_multiplier, max_grad_norm=max_grad_norm
        )
        privacy_engine.attach(optim)
    else:
        privacy_engine = None

    # Training
    losses, test_accs, test_losses, epsilons, alphas, epoch_times = train(BATCH_SIZE, EPOCHS, DELTA,
                                                                          model, torch,
                                                                          optim, loss_function, 
                                                                          train_data, train_labels, 
                                                                          test_data, test_labels, 
                                                                          [1, 64, 64], device, privacy_engine)

    # Validation
    val_acc, val_loss = test(model, loss_function, torch, val_data, val_labels, device)

    print(f'Validation Accuracy: {val_acc} ---- Validation Loss: {val_loss}')

    # Tracking all interesting variables and results in .csv file
    if TRACKING:
        d = {
            'model': MODEL,
            'dataset': DATASET,
            'batch_size': BATCH_SIZE,
            'epochs': EPOCHS,
            'learning_rate': learning_rate,
            'train_sample_size': SAMPLE_SIZE,
            'test_sample_size': len(test_data),
            'val_sample_size': len(val_data),
            'delta': DELTA,
            'noise_multiplier': noise_multiplier,
            'max_grad_norm': max_grad_norm,
            'dp_used': DP,
            'epsilons': epsilons,
            'alphas': alphas,
            'train_losses': losses,
            'test_accs': test_accs,
            'test_losses': test_losses,
            'val_acc': val_acc,
            'val_loss': val_loss,
            'epoch_times': epoch_times
        }      
        df = pd.read_csv('./Results/1DS.csv')
        df = df.append(d, ignore_index=True)
        df.to_csv('./Results/1DS.csv', index=False)

### Gridsearch

In [5]:
import itertools

def gridsearch(lrs, noises, norms):
    grid = itertools.product(lrs, noises, norms)
    num = 0
    
    if DP:
        for learning_rate, noise_multiplier, max_grad_norm in grid:
            num += 1
            print(f'################################# RUN No. {num} #################################')
            run(learning_rate, noise_multiplier, max_grad_norm)
            
    else:
        for learning_rate in lrs:
            num += 1
            print(f'################################# RUN No. {num} #################################')
            run(learning_rate, 0 , 0)
            
    
lrs = [0.0025, 0.001, 0.0015, 0.002, 0.003]
noises = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]
norms = [1.0, 5.0, 10.0]

gridsearch(lrs, noises, norms)

################################# RUN No. 1 #################################


A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.


###### Epoch 1 ######


Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


(ε = 525.69, δ = 0.0001) for α = 1.1
Test Accuracy: 0.1262135922330097 ---- Test Loss: 1.7896469831466675
Epoch time: 3.3233845233917236 seconds
###### Epoch 2 ######
(ε = 959.28, δ = 0.0001) for α = 1.1
Test Accuracy: 0.46601941747572817 ---- Test Loss: 2.280076503753662
Epoch time: 0.9972844123840332 seconds
###### Epoch 3 ######
(ε = 1392.86, δ = 0.0001) for α = 1.1
Test Accuracy: 0.6407766990291263 ---- Test Loss: 1.707815170288086
Epoch time: 1.0009212493896484 seconds
###### Epoch 4 ######
(ε = 1826.45, δ = 0.0001) for α = 1.1
Test Accuracy: 0.8543689320388349 ---- Test Loss: 0.7431210279464722
Epoch time: 1.024198055267334 seconds
###### Epoch 5 ######
(ε = 2260.04, δ = 0.0001) for α = 1.1
Test Accuracy: 0.8786407766990292 ---- Test Loss: 0.8386223912239075
Epoch time: 1.0039129257202148 seconds
###### Epoch 6 ######
(ε = 2693.62, δ = 0.0001) for α = 1.1
Test Accuracy: 0.8980582524271845 ---- Test Loss: 0.46400049328804016
Epoch time: 1.0589630603790283 seconds
###### Epoch 7 ##

...

(ε = 0.42, δ = 0.0001) for α = 39.0
Test Accuracy: 0.1262135922330097 ---- Test Loss: 1.79460871219635
Epoch time: 1.0399467945098877 seconds
###### Epoch 2 ######
(ε = 0.56, δ = 0.0001) for α = 31.0
Test Accuracy: 0.15048543689320387 ---- Test Loss: 1.7914085388183594
Epoch time: 1.030937910079956 seconds
###### Epoch 3 ######
(ε = 0.67, δ = 0.0001) for α = 26.0
Test Accuracy: 0.2766990291262136 ---- Test Loss: 1.768852710723877
Epoch time: 1.0509588718414307 seconds
###### Epoch 4 ######
(ε = 0.77, δ = 0.0001) for α = 23.0
Test Accuracy: 0.2524271844660194 ---- Test Loss: 1.687999963760376
Epoch time: 1.0257399082183838 seconds
###### Epoch 5 ######
(ε = 0.85, δ = 0.0001) for α = 21.0
Test Accuracy: 0.5097087378640777 ---- Test Loss: 1.6196027994155884
Epoch time: 1.0239315032958984 seconds
###### Epoch 6 ######
(ε = 0.93, δ = 0.0001) for α = 20.0
Test Accuracy: 0.6067961165048543 ---- Test Loss: 1.3952592611312866
Epoch time: 1.024435043334961 seconds
###### Epoch 7 ######
(ε = 1.00

A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


(ε = 0.34, δ = 0.0001) for α = 48.0
Test Accuracy: 0.2524271844660194 ---- Test Loss: 1.7885478734970093
Epoch time: 1.1240227222442627 seconds
###### Epoch 2 ######
(ε = 0.47, δ = 0.0001) for α = 37.0
Test Accuracy: 0.2524271844660194 ---- Test Loss: 1.7743003368377686
Epoch time: 1.1050050258636475 seconds
###### Epoch 3 ######
(ε = 0.56, δ = 0.0001) for α = 32.0
Test Accuracy: 0.2524271844660194 ---- Test Loss: 1.7214372158050537
Epoch time: 1.1015064716339111 seconds
###### Epoch 4 ######
(ε = 0.64, δ = 0.0001) for α = 28.0
Test Accuracy: 0.1262135922330097 ---- Test Loss: 1.7266572713851929
Epoch time: 1.1020045280456543 seconds
###### Epoch 5 ######
(ε = 0.72, δ = 0.0001) for α = 26.0
Test Accuracy: 0.5533980582524272 ---- Test Loss: 1.650336503982544
Epoch time: 1.0594663619995117 seconds
###### Epoch 6 ######
(ε = 0.78, δ = 0.0001) for α = 24.0
Test Accuracy: 0.470873786407767 ---- Test Loss: 1.609275460243225
Epoch time: 1.046952486038208 seconds
###### Epoch 7 ######
(ε = 0.8

A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


(ε = 0.29, δ = 0.0001) for α = 57.0
Test Accuracy: 0.07766990291262135 ---- Test Loss: 1.790595293045044
Epoch time: 1.0459508895874023 seconds
###### Epoch 2 ######
(ε = 0.40, δ = 0.0001) for α = 44.0
Test Accuracy: 0.36893203883495146 ---- Test Loss: 1.783674955368042
Epoch time: 1.065969705581665 seconds
###### Epoch 3 ######
(ε = 0.48, δ = 0.0001) for α = 37.0
Test Accuracy: 0.3786407766990291 ---- Test Loss: 1.7646819353103638
Epoch time: 1.0284361839294434 seconds
###### Epoch 4 ######
(ε = 0.55, δ = 0.0001) for α = 33.0
Test Accuracy: 0.4174757281553398 ---- Test Loss: 1.7300705909729004
Epoch time: 1.0684576034545898 seconds
###### Epoch 5 ######
(ε = 0.62, δ = 0.0001) for α = 30.0
Test Accuracy: 0.1262135922330097 ---- Test Loss: 1.7310850620269775
Epoch time: 1.0239310264587402 seconds
###### Epoch 6 ######
(ε = 0.67, δ = 0.0001) for α = 27.0
Test Accuracy: 0.39805825242718446 ---- Test Loss: 1.616809606552124
Epoch time: 1.025933027267456 seconds
###### Epoch 7 ######
(ε = 0